# Comparison between QuickNXS version 1 and 2

- Comparison of reflectivity using SANGLE or DANGLE
- Comparison of off-specular using "smoothing"
- Comparison of projections of off-specular data on each axis
- Comparison of specular reflectivity (using DANGLE)

Note: QuickNXS v1 uses the whole TOF range and v2 trims the TOF distribution to match a wavelength band of 3.2 A, which is most of the neutrons.

In [1]:
import os
import numpy as np
import scipy.stats
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings('ignore', module='numpy')
warnings.filterwarnings('ignore')

In [2]:
import sys
sys.path.append("/SNS/users/m2d/git/reflectivity_ui/test/notebooks")
from plot_utils import plot1d, plot_heatmap

In [40]:
def load_data(file_path, ncols=3):
    offspec_data = np.loadtxt(file_path)

    unique_qy = np.unique(offspec_data.T[0])
    qybins = len(unique_qy)
    
    unique_qz = np.unique(offspec_data.T[1])
    qzbins = len(unique_qz)

    print("\n\nFile: %s" % file_path)
    print("Binning: %s %s [%s]" %(qybins, qzbins, qybins*qzbins))
    fd = open(file_path, 'r')
    first_list = fd.readlines()
    print(first_list[0]+first_list[1]+first_list[2])
    fd.close()

    off = np.reshape(offspec_data, (qzbins, qybins, ncols))

    #qz = (off.T[1]).T[0]
    #qy = (off.T[0])[0]
    qy = (off.T[0]).T[0]
    qz = (off.T[1])[0]
    #print "Qy [%s] %s - %s" % (len(qy), np.min(qy), np.max(qy))
    #print "Qz [%s] %s - %s" % (len(qz), np.min(qz), np.max(qz))

    quick_intensity = off.T[2].T
    
    scale = 1
    quick_intensity /= scale
    quick_intensity[quick_intensity<1.0e-9] = 1.0e-9

    return qy, qz, quick_intensity



## Comparisons between reflectivity calculations using SANGLE or DANGLE

In [59]:
sangle_v2_path = '/SNS/users/m2d/git/reflectivity_ui/test/comparison/v2/REF_M_30806_Specular_Off_Off_SANGLE.dat'
dangle_v2_path = '/SNS/users/m2d/git/reflectivity_ui/test/comparison/v2/REF_M_30806_Specular_Off_Off_DANGLE.dat'
sangle_v2 = np.loadtxt(sangle_v2_path).T
dangle_v2 = np.loadtxt(dangle_v2_path).T
sangle_v1_path = '/SNS/users/m2d/git/reflectivity_ui/test/comparison/v1/REF_M_30806_Specular_Off_Off_SANGLE.dat'
dangle_v1_path = '/SNS/users/m2d/git/reflectivity_ui/test/comparison/v1/REF_M_30806_Specular_Off_Off_DANGLE.dat'
sangle_v1 = np.loadtxt(sangle_v1_path).T
dangle_v1 = np.loadtxt(dangle_v1_path).T

plot1d([[sangle_v2[0], sangle_v2[1], sangle_v2[2]],
        [dangle_v2[0], dangle_v2[1], dangle_v2[2]],
        [sangle_v1[0], sangle_v1[1], sangle_v1[2]],
        [dangle_v1[0], dangle_v1[1], dangle_v1[2]]
       ],
       data_names=['SANGLE v2', 'DANGLE v2','SANGLE v1', 'DANGLE v1'],
      y_log=True, x_log=True, x_title='Q', y_title='R')

## Off-specular, with "smoothing"

In [65]:
filepath = '/SNS/users/m2d/git/reflectivity_ui/test/comparison/v1/REF_M_30806+30807+30808_OffSpecSmooth_Off_Off_set_1.dat'
dk, qz, off_spec = load_data(filepath, ncols=3)

plot_heatmap(dk, qz, np.log(off_spec), x_title='kf_z-ki_i', y_title='Qz', surface=False,
                 x_log=False, y_log=False)



File: /SNS/users/m2d/git/reflectivity_ui/test/comparison/v1/REF_M_30806+30807+30808_OffSpecSmooth_Off_Off_set_1.dat
Binning: 84 282 [23688]
# Datafile created by QuickNXS 1.1.6
# Date: 2019-01-09 13:30:17
# Type: OffSpecSmooth



In [66]:
filepath = '/SNS/users/m2d/git/reflectivity_ui/test/comparison/v2/REF_M_30806+30807+30808_OffSpecSmooth_Off_Off_set_1.dat'
dk_v2, qz_v2, off_spec_v2 = load_data(filepath, ncols=3)

plot_heatmap(qz, dk, np.log(off_spec), x_title='kf_z-ki_i', y_title='Qz', surface=False,
                 x_log=False, y_log=False)




File: /SNS/users/m2d/git/reflectivity_ui/test/comparison/v2/REF_M_30806+30807+30808_OffSpecSmooth_Off_Off_set_1.dat
Binning: 84 282 [23688]
# Datafile created by QuickNXS 2.0.31
# Datafile created using Mantid 3.13.20190108.2020
# Date: 2019-01-09 14:30:36



## Projections on each axis

In [68]:
cut_v1 = [qz, off_spec.sum(0)]
cut_v2 = [qz_v2, off_spec_v2.sum(0)]

cut_y_v1 = [dk, off_spec.sum(1)]
cut_y_v2 = [dk_v2, off_spec_v2.sum(1)]

plot1d([cut_v1,cut_v2, cut_y_v1, cut_y_v2],
       x_title="Projection axis (see legend)", y_title='Intensity',
       data_names=['qz v1', 'qz v2', 'kf_z-ki_i v1', 'kf_z-ki_i v2'], y_log=True)

## Comparison of specular reflectivity

The compatation uses DANGLE to compute the scattering angle.

In [64]:
# Using DANGLE in both cases
v2_path = '/SNS/users/m2d/git/reflectivity_ui/test/comparison/v2/REF_M_30806+30807+30808_Specular_Off_Off_set_1.dat'
v2 = np.loadtxt(v2_path).T
v1_path = '/SNS/users/m2d/git/reflectivity_ui/test/comparison/v1/REF_M_30806+30807+30808_Specular_Off_Off_set_1.dat'
v1 = np.loadtxt(v1_path).T

plot1d([[v2[0], v2[1], v2[2]],
        [v1[0], v1[1], v1[2]],
       ],
       data_names=['v2', 'v1'],
       x_title='Q', y_title='R',
      y_log=True, x_log=True)